In [1]:
import re
import spacy
import textract
import os
from spacy.matcher import Matcher
import glob
import os
import random
import pandas as pd
import json

resumes_dict = {
    'resume_id':
    {                  #filename for now but to be replaced with unique id
        'name': ('first name', 'last name'),
        'email': ('email', 'position_tuple'),
        'phone': ('phone', 'position_tuple'),
        'dob': ('dob', 'position_tuple'),
        'address': ('address', 'position_tuple'),
        'skills': [('skill1', 'position_tuple1'), ('skill2', 'position_tuple2')],
    }                     
}

In [2]:
file_list = glob.glob(os.path.join(os.getcwd(), "Fw__Sample_resumes_", "*.docx"))

In [3]:
file_list

['/home/kr/notebooks/qualifyde-resume-parsing/Fw__Sample_resumes_/Resume1_UI.docx',
 '/home/kr/notebooks/qualifyde-resume-parsing/Fw__Sample_resumes_/Resume2_Java_full_stack.docx',
 '/home/kr/notebooks/qualifyde-resume-parsing/Fw__Sample_resumes_/Resume4_Devops_engg.docx',
 '/home/kr/notebooks/qualifyde-resume-parsing/Fw__Sample_resumes_/Resume_11_Dotnet_Developer_.docx',
 '/home/kr/notebooks/qualifyde-resume-parsing/Fw__Sample_resumes_/Resume_12_backend_developer_.docx',
 '/home/kr/notebooks/qualifyde-resume-parsing/Fw__Sample_resumes_/Resume_6_data_engg.docx',
 '/home/kr/notebooks/qualifyde-resume-parsing/Fw__Sample_resumes_/Resume_9_cyber_security_.docx',
 '/home/kr/notebooks/qualifyde-resume-parsing/Fw__Sample_resumes_/resume_8_service_now.docx',
 '/home/kr/notebooks/qualifyde-resume-parsing/Fw__Sample_resumes_/Resume-Dileep2020 (1).docx']

In [65]:
class parse:
    def __init__(self, file_list):
        self.file_list = file_list
        self.resumes = [textract.process(file_path).decode() for file_path in file_list]
        self.nlp_name_loc = spacy.load('en_core_web_trf')
        self.nlp_skills = spacy.load('en_core_web_md')
        self.nlp_edu = spacy.load('en_core_web_sm')
    def extract(self):
        edu_company = [
                'BE',
                'B.E.', 
                'B.E', 
                'BS',
                'B.S',
                'B.Com',
                'BCA',
                'ME',
                'M.E', 
                'M.E.', 
                'M.S',
                'B.com',
                '10',
                '10+2',
                'BTECH', 
                'B.TECH', 
                'M.TECH', 
                'MTECH', 
                'SSC', 
                'HSC', 
                'C.B.S.E',
                'CBSE',
                'ICSE', 
                'X', 
                'XII',
                '10th',
                '12th',
                ' 10th',
                ' 12th',
                'College',
                'University',
                'School',
                'Pvt',
                'Company'
            ]
        ph_list, email_list, dob_list, names_list, location_list, skills_list, skillset_exp = [], [], [], [], [], [], []
        
        for resume in self.resumes:
            items = re.finditer(r'\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4}', resume)
            ph_span = [(resume[item.span()[0]:item.span()[1]], item.span()) for item in items]
            ph_list.append(ph_span)
            items = re.finditer(r'[\w\.-]+@[\w\.-]+', resume)
            em_span = [(resume[item.span()[0]:item.span()[1]], item.span()) for item in items]
            email_list.append(em_span)
            result2 = re.finditer(r"[\d]{1,2}-[\d]{1,2}-[\d]{4}", resume) # DD-MM-YYYY
            dob_span = [(resume[item.span()[0]:item.span()[1]], item.span()) for item in result2]
            dob_list.append(dob_span)
            doc_nmlc = self.nlp_name_loc(resume)
            doc_skills = self.nlp_skills(resume)
            list_ls, list_ns, list_ss = [], [], []
            for ent in doc_nmlc.ents:
                if ent.label_ == 'PERSON':
                    es_names = (ent.text, ent.start_char, ent.end_char)
                    list_ns.append(es_names)
                if ent.label_ in ['GPE', 'LOC']:
                    es_locs = (ent.text, ent.start_char, ent.end_char)
                    list_ls.append(es_locs) 
            for ent in doc_skills.ents:
                if ent.label_ in ['PRODUCT', 'ORG']:
                    s_span = (ent.text, ent.start_char, ent.end_char)
                    list_ss.append(s_span)
            location_list.append(list_ls)
            names_list.append(list_ns)
            skillset_exp.append(list_ss)
            noun_chunks = doc_skills.noun_chunks
            # tokens = [token.text for token in nlp_text if not token.is_stop]
            tokens = [token.text for token in doc_skills if token.is_stop]
            data = pd.read_csv("skills.csv") 
            skills = list(data.columns.values)

            skillset = []
            for token in tokens:
                if token.lower() in skills:
                    skillset.append(token)

            for token in noun_chunks:
                token = token.text.lower().strip()
                if token in skills:
                    skillset.append(token)

            st = list(set([i.capitalize() for i in set([i.lower() for i in skillset])]))
            skills_list.append(st)
        
            
        return ph_list, email_list, dob_list, names_list, location_list, skill_list, skillset_exp
    
    def dictify(self):
        dict_ = {}
        ph_list, email_list, dob_list, names_list, location_list, skills_list, skillset_exp = self.extract()
        for idx, ph, email, dob, name, location, skills, skills_exp in zip(
            [file.split('.')[-2].split('/')[-1] for file in self.file_list], 
            ph_list, 
            email_list, 
            dob_list, 
            names_list, 
            location_list, 
            skills_list,
            skillset_exp
        ):
            dict_[idx] = {}
            if len(dob):
                dict_[idx]['dob'] = dob[0]
            else:
                dict_[idx]['dob'] = (None, None)
            if len(name):
                dict_[idx]['name'] = name[0]
            else:
                dict_[idx]['name'] = (None, None)
            if len(email): 
                dict_[idx]['email'] = email[0]
            else:
                dict_[idx]['email'] = (None, None)
            if len(ph):
                dict_[idx]['phone'] = ph[0]
            else:
                dict_[idx]['phone'] = (None, None)
            if len(location):
                dict_[idx]['location'] = location[0]
            else:
                dict_[idx]['location'] = (None, None)
            dict_[idx]['skills'] = skills
            dict_[idx]['skills_exp'] = skills_exp
        return dict_     

In [66]:
#nlp = spacy.load('en_core_web_sm')
p = parse(file_list)

In [67]:
%%time
#ph_list, email_list, dob_list, names_list, location_list, skills_list, ents_ = p.extract()
resume_dict = p.dictify()

CPU times: user 5min 30s, sys: 41.1 s, total: 6min 11s
Wall time: 1min 7s


In [68]:
with open('parsed_output2.json', 'w') as f:
    json.dump(resume_dict, f)

In [79]:
for skill in skill_list:
    print(skill)

('INDIA • 9949171669 • 9985525299\n\n\n\n\t\tBusiness Architect', 109, 164)
('Business Analyst\n\n\t\tDesign Thinking\n\n\t\tProduct Concept', 250, 304)
('Project Management\n\n\t\tAgile-Scrum & Waterfall\n\n\t\tSoftware Development Life Cycle Management\n\n\t\tBusiness Process Management\n\n\t\tChange Management\n\n\t\tStakeholder Management\n\n\t\tEstimations', 358, 542)
('Delivery & Release Planning', 544, 571)
('Micro Mobility', 1011, 1025)
('Directors / Managers', 1098, 1118)
('Conceptualization, Design Thinking', 1292, 1326)
('Operations and Marketing', 1492, 1516)
('Project Management and Team Collaboration for Projects and Products', 1643, 1710)
('PMBOK) &', 2270, 2278)
('SDLC', 2586, 2590)
('Project', 2595, 2602)
('Agile', 2635, 2640)
('Agile', 3219, 3224)
('Scrum-centric', 3257, 3270)
('the Business and Operations', 3528, 3555)
('Scrum Master', 4076, 4088)
('UPSC Civil Services Examinations', 4184, 4216)
('Consultancy', 4262, 4273)
('Content', 4314, 4321)
('Learning Managemen

In [72]:
resume_dict['Resume-Dileep2020 (1)']['skills']

('Project Management and Team Collaboration for Projects and Products',
 1643,
 1710)

In [77]:
skills = resume_dict['Resume-Dileep2020 (1)']['skills_exp']

new_skill_list = [skill for skill in skills if not any([exclusion.lower() in skill[0].lower() for exclusion in edu_company])]

In [89]:
from nltk.corpus import stopwords


def extract_education(resume_text):
    STOPWORDS = set(stopwords.words('english'))
    
    nlp_text = nlp(resume_text)
    nlp_text = [sent.text.strip() for sent in nlp_text.sents]

    edu = {}
    for index, text in enumerate(nlp_text):
        for tex in text.split():
            tex = re.sub(r'[?|$|.|!|,]', r'', tex)
            if tex.upper() in EDUCATION and tex not in STOPWORDS:
                edu[tex] = text + nlp_text[index]

    education = []
    for key in edu.keys():
        year = re.search(re.compile(r'(((20|19)(\d{2})))'), edu[key])
        if year:
            education.append((key, ''.join(year[0])))
        else:
            education.append(key)
    return education

In [91]:
l_ = extract_education(textract.process(file_list[-1]).decode())

In [92]:
l_

[('MTech', '2009'), ('BTech', '2009')]

In [93]:
'en' in 'eng'

True

In [54]:
not any([0, ''])

True